In [1]:
import glob, os
import json
import pydicom

from MRLCinema.readcine.readcines import readcines
from MRLCinema.patient_data import prescription_ds, rtss_frame_of_reference_ds
from MRLCinema.extract_motion import extract_motion
from MRLCinema.report import create_report
from U2Dose.dicomio.rtstruct import RtStruct

motion_management_data_path = '/mnt/Q'
motion_management_result_path = '/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/MotionManagement'
patient_data_root = f'/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/PATIENT_DATA'
patient_data_archive_root = f'/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/Patient_Data_Archive'

## Determine all prostate patients that is schedulaed for 7fx -> 42.7 Gy

In [15]:

prostate_patients = {} #[] 7set()
other_patients = {}
recti_patients = {}

prostate_prescribed_dose = 42.7
prostate_number_of_fractions = 7

def add_to_dict_set(dict, patient_dir, fraction_dir):
    if patient_dir not in dict:
        dict[patient_dir] = set()
    
    if fraction_dir in dict[patient_dir]:
        print(f"Warning: Fraction directory {fraction_dir} already exists for patient {patient_dir}")
        raise ValueError(f"Fraction directory {fraction_dir} already exists for patient {patient_dir}")

    dict[patient_dir].add(fraction_dir)

def set_dict(dict, key, value):
    if key in dict:
        print(f"Warning: Key {key} already exists in the dictionary")
        raise ValueError(f"Key {key} already exists in the dictionary")
    dict[key] = value

def count_fractions(dict):
    count = 0
    for patient_dir, fraction_dirs in dict.items():
        count += len(fraction_dirs)
    return count

def find_patients(patient_data_root, prostate_patients, recti_patients, other_patients):

    for patient_dir in os.listdir(patient_data_root):

        if not os.path.isdir(os.path.join(patient_data_root, patient_dir)):
            continue

        if patient_dir.startswith('MRL'):
            continue

        if not patient_dir.startswith('19'):
            continue
        
        for fraction_dir in os.listdir(os.path.join(patient_data_root, patient_dir)):
            
            if not os.path.isdir(os.path.join(patient_data_root, patient_dir, fraction_dir)):
                continue

            # Check if the fraction directory contains 'ADT' but not 'QA'                
            if ('ADT' in fraction_dir) and ('QA' not in fraction_dir):

                rplan_filenames = glob.glob(os.path.join(patient_data_root, patient_dir, fraction_dir, 'RP*.dcm'))
                
                if len(rplan_filenames) > 0:
                    ds = pydicom.dcmread(rplan_filenames[0], force=True)
                    number_of_fractions, prescription_dose = prescription_ds(ds)
                    if (number_of_fractions == prostate_number_of_fractions) and (prescription_dose == prostate_prescribed_dose):
                        add_to_dict_set(prostate_patients, patient_dir, fraction_dir)
                    elif (number_of_fractions == 5) and (prescription_dose == 25.0):
                        add_to_dict_set(recti_patients, patient_dir, fraction_dir)
                    else:   
                        add_to_dict_set(other_patients, patient_dir, fraction_dir)        
                

    return prostate_patients, recti_patients, other_patients         
                            

find_patients(patient_data_root, prostate_patients, recti_patients, other_patients)
find_patients(patient_data_archive_root, prostate_patients, recti_patients, other_patients)

print(len(prostate_patients), len(recti_patients), len(other_patients))

import pprint
pprint.pprint(prostate_patients)

print(count_fractions(prostate_patients))

83 14 52
{'193211030873': {'10MRL1ADT01',
                  '10MRL1ADT02',
                  '10MRL1ADT03',
                  '10MRL1ADT05',
                  '10MRL1ADT06',
                  '10MRL1ADT07',
                  '10MRL1ADT08'},
 '194202130656': {'12MRL1ADT01',
                  '12MRL1ADT02',
                  '12MRL1ADT03',
                  '12MRL1ADT04',
                  '12MRL1ADT05',
                  '12MRL1ADT06',
                  '12MRL1ADT07'},
 '194204238531': {'15MRL1ADT01',
                  '15MRL1ADT02',
                  '15MRL1ADT03',
                  '15MRL1ADT04',
                  '15MRL1ADT05',
                  '15MRL1ADT06',
                  '15MRL1ADT07'},
 '194205201496': {'10MRL1ADT01',
                  '10MRL1ADT03',
                  '10MRL1ADT04',
                  '10MRL1ADT05',
                  '10MRL1ADT06',
                  '10MRL1ADT07',
                  '10MRL1ADT09'},
 '194401087517': {'11MRL1ADT01',
                  '11MRL1ADT02

### See how many has been analysed (and not)

In [16]:
missing_analysis = {}

for patient in prostate_patients:

    for fraction in prostate_patients[patient]:  
        
        if not os.path.isfile(os.path.join(motion_management_result_path, f'{patient}_{fraction}_cine_motion_analysis.json')):
            add_to_dict_set(missing_analysis, patient, fraction)
        #else:
        #    print(f'Found {patient}_{fraction}_cine_motion_analysis.json')
            

len(missing_analysis)
#pprint.pprint(missing_analysis)
print(177, 576)
count_fractions(missing_analysis), count_fractions(prostate_patients)


177 576


(153, 576)

In [8]:
#
# Create list of that exist in the motion management data path, loop over motion management data and extract patient IDs
#
from MRLCinema.patient_data import read_cine_patient_ID, find_cine_frame_of_reference

motion_management_patient_FoR = {} 
motion_management_FoR_directory = {} 

for mm_dir in os.listdir(motion_management_data_path):
    path = os.path.join(motion_management_data_path, mm_dir)
    if not os.path.isdir(path):
        continue
    try:
        # Attempt to read the patient ID from the directory
        patient_ID = read_cine_patient_ID(path)
        frame_of_ref = find_cine_frame_of_reference(path)
        if patient_ID.startswith('19'):
            add_to_dict_set(motion_management_patient_FoR, patient_ID, frame_of_ref)
            set_dict(motion_management_FoR_directory, frame_of_ref, path)
    except Exception as e:
        #print(f"Error reading patient ID from {path}: {e}")
        continue

pprint.pprint(motion_management_FoR_directory)


{'1.3.46.670589.11.79101.5.0.10092.2022110709013772001': '/mnt/Q/1.3.46.670589.11.79101.5.0.15444.2022110709013617002',
 '1.3.46.670589.11.79101.5.0.10092.2022110710062592002': '/mnt/Q/1.3.46.670589.11.79101.5.0.15444.2022110710084504006',
 '1.3.46.670589.11.79101.5.0.10092.2022110710552316003': '/mnt/Q/1.3.46.670589.11.79101.5.0.15444.2022110710552094008',
 '1.3.46.670589.11.79101.5.0.10312.2022081510120142001': '/mnt/Q/1.3.46.670589.11.79101.5.0.204.2022081510120023002',
 '1.3.46.670589.11.79101.5.0.10392.2024021209385145001': '/mnt/Q/1.3.46.670589.11.79101.5.0.15336.2024021209384945002',
 '1.3.46.670589.11.79101.5.0.10468.2022110209150574001': '/mnt/Q/1.3.46.670589.11.79101.5.0.16424.2022110209150432002',
 '1.3.46.670589.11.79101.5.0.10468.2022110210254507002': '/mnt/Q/1.3.46.670589.11.79101.5.0.16424.2022110210254264004',
 '1.3.46.670589.11.79101.5.0.10468.2022110211081965003': '/mnt/Q/1.3.46.670589.11.79101.5.0.16424.2022110211081754006',
 '1.3.46.670589.11.79101.5.0.1048.20240524

In [9]:
#
# Compare the motion management patients with the missing prostate patients
# 
def rtstruct_filenames(patient, fraction):
    filenames = glob.glob(os.path.join(patient_data_archive_root, patient, fraction, 'RS*.dcm'))
    if len(filenames) == 0:
        filenames = glob.glob(os.path.join(patient_data_archive_root, patient, fraction, 'RS*.dcm')) 
    return filenames

errors = {} 

for missing_patient in missing_analysis:
    if missing_patient in motion_management_patient_FoR:
        
        #print(f'Found {missing_patient} in motion management data')

        # try to run analysis! 
        for fraction in missing_analysis[missing_patient]:
            
            try:
                rtss_filenames = rtstruct_filenames(missing_patient, fraction)
                if len(rtss_filenames) == 0:
                    errors[f'{missing_patient} {fraction}'] = 'No RTSTRUCT found' 
                    continue

                rtss = RtStruct(rtss_filenames[0], force=True)
                frame_of_ref_rtss = rtss_frame_of_reference_ds(rtss._ds)
                if frame_of_ref_rtss in motion_management_patient_FoR[missing_patient]:
                    
                    print(f'Frame of reference {frame_of_ref_rtss} found for {missing_patient} {fraction} - DO ANALYSIS')
                    rtss.parse()

                    #
                    # Read the cine data
                    #
                    cine_directory = os.path.join(motion_management_FoR_directory[frame_of_ref_rtss], 'TwoDImages')
                    #start = time.time()
                    cines = readcines(cine_directory, max_n=2000)

                    #
                    # Extract the motion
                    #   
                    displacements, times = extract_motion(cines, rtss)

                    #
                    # Create the report, write to fraction directory
                    # { } []
                    report = create_report(missing_patient, cine_directory, fraction, [prostate_prescribed_dose, prostate_number_of_fractions], times, displacements)
                    report_filename = os.path.join(motion_management_result_path, f'{missing_patient}_{fraction}_cine_motion_analysis.json')
                    with open(report_filename, 'w') as f:
                        json.dump(report, f, indent=4)
                        print(f'Wrote report to {report_filename}')

            except Exception as e:
                errors[f'{missing_patient} {fraction}'] = f'Exception: {e}' 
                continue
                
    else:
        errors[f'{missing_patient} {fraction}'] = f'Not found {missing_patient} in motion management data' 
        #pprint.pprint(missing_analysis[missing])

Frame of reference 1.3.46.670589.11.79101.5.0.7112.2022040813083810002 found for 194603102957 15MRL1ADT03 - DO ANALYSIS
Frame of reference 1.3.46.670589.11.79101.5.0.12524.2022041113041600001 found for 194603102957 15MRL1ADT04 - DO ANALYSIS
Frame of reference 1.3.46.670589.11.79101.5.0.7672.2022040413252312002 found for 194603102957 15MRL1ADT01 - DO ANALYSIS
Frame of reference 1.3.46.670589.11.79101.5.0.8528.2022041413130713001 found for 194603102957 15MRL1ADT05 - DO ANALYSIS


In [12]:
pprint.pprint(errors)
len(errors)

{'193211030873 10MRL1ADT03': 'No RTSTRUCT found',
 '194204238531 11MRL1ADT08': 'Not found 194204238531 in motion management data',
 '194401087517 11MRL1ADT08': 'Not found 194401087517 in motion management data',
 '194409238252 10MRL1ADT08': 'No RTSTRUCT found',
 '194503246458 10MRL1ADT04': 'No RTSTRUCT found',
 '194503246458 10MRL1ADT06': 'No RTSTRUCT found',
 '194503246458 10MRL1ADT10': 'No RTSTRUCT found',
 '194505041238 11MRL1ADT07': 'No RTSTRUCT found',
 '194507298711 11MRL1ADT08': 'Not found 194507298711 in motion management data',
 '194509136950 11MRL1ADT08': 'Not found 194509136950 in motion management data',
 '194509301232 12MRL1ADT03': 'No RTSTRUCT found',
 '194510121439 11MRL1ADT08': 'Not found 194510121439 in motion management data',
 '194512017536 11MRL1ADT08': 'Not found 194512017536 in motion management data',
 '194601251418 11MRL1ADT08': 'Not found 194601251418 in motion management data',
 '194602065213 11MRL1ADT08': 'Not found 194602065213 in motion management data',
 '

43

In [17]:
missing_analysis

{'193211030873': {'10MRL1ADT03'},
 '194409238252': {'10MRL1ADT08'},
 '194503246458': {'10MRL1ADT04', '10MRL1ADT06', '10MRL1ADT10'},
 '194505041238': {'11MRL1ADT07'},
 '194509301232': {'12MRL1ADT03'},
 '194706127331': {'10MRL1ADT06'},
 '194805016955': {'10MRL1ADT07', '10MRL1ADT08'},
 '194807111135': {'10MRL1ADT05'},
 '194810237893': {'10MRL1ADT02', '10MRL1ADT03'},
 '194904158112': {'10MRL1ADT11'},
 '195201231577': {'10MRL1ADT05', '10MRL1ADT06'},
 '195408022399': {'10MRL1ADT03'},
 '196101161179': {'11MRL1ADT03'},
 '196209251070': {'11MRL1ADT08'},
 '196612157252': {'11MRL1ADT08'},
 '194204238531': {'15MRL1ADT01',
  '15MRL1ADT02',
  '15MRL1ADT03',
  '15MRL1ADT04',
  '15MRL1ADT05',
  '15MRL1ADT06',
  '15MRL1ADT07'},
 '194401087517': {'11MRL1ADT01',
  '11MRL1ADT02',
  '11MRL1ADT03',
  '11MRL1ADT04',
  '11MRL1ADT05',
  '11MRL1ADT06',
  '11MRL1ADT07'},
 '194507298711': {'12MRL1ADT01',
  '12MRL1ADT02',
  '12MRL1ADT03',
  '12MRL1ADT04',
  '12MRL1ADT05',
  '12MRL1ADT06',
  '12MRL1ADT07'},
 '19450